In [1]:
import nltk
import re
import numpy as np
import os
from scipy.spatial.distance import cosine

reemplazo_acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u'}
#nltk.download("punkt")
#cargo los stopwords
with open("spanish.txt","r",encoding="utf-8") as sp:
    stop_list=[word.strip() for word in sp.readlines()]
#cargo el libro


## Funciones

No uso todas. Creo que la mejor solucion es con objetos alucina.

In [2]:
def prepro(strp):
    limpio = re.sub(r'[áéíóú]', lambda x: reemplazo_acentos[x.group()], strp)
    # Tokenización y eliminación de puntuación
    tokens = nltk.word_tokenize(re.sub(r'[^\w\s]', '', limpio), language="spanish")
    tokens=[token for token in tokens if token not in stop_list]
    return tokens

In [3]:
class Vector():
    def __init__(self, corpus) -> None:

        self.corpus = list(corpus.values())
        self.files= {x:i for i,x in enumerate(list(corpus.keys()))}

        self.dicc = {}
        self.set_dic()

        self.n_doc = len(self.files)
        self.n_terms = len(self.dicc)

        self.tf = np.zeros((self.n_doc, self.n_terms), dtype=int)
        self.idf = np.zeros(self.n_terms)
        self.tf_idf = np.zeros((self.n_doc, self.n_terms), dtype=float)
        self.tf_idf_keys={}    

    def set_dic(self):
        tempdic=[]
        for v in self.corpus:
            tempdic.extend(v)
        self.dicc={x:i for i, x in enumerate(set(tempdic))}
        
    def get_idf(self):
        self.idf = np.count_nonzero(self.tf, axis=0)

    def get_tf_idf(self):
        self.tf_idf=self.tf*np.log2(self.n_doc/self.idf)

    def get_tf(self,mode='glob',topn = False, minv = False):
        res_dic=[]       
        #para cada elemento en corpus 
        for i,v in enumerate(self.corpus):
            #llevamos cuenta de las palabras. Siguiendo el diccionario general
            dicc=dict.fromkeys(self.dicc.keys(), 0)

            #contea las palabras en corpus
            for word in v:
                dicc[word]=dicc.get(word,0) + 1
            
            #crea el array
            conteo=np.array(list(dicc.values()))

            match mode:
                case 'glob':
                    pass

                case 'topn':
                    #como hacemos en el ordenado
                    #filtramos
                    dicc=dict(sorted(dicc.items(), key=lambda item: item[1],reverse=True))
                    dicc=list(dicc)[:topn]
                    res_dic.extend(dicc)

                case 'minv':
                    dicc={k:v for k,v in dicc.items() if v>minv}
                    dicc=list(dicc)
                    res_dic.extend(dicc)
                    #filtramos, pero ahora minv
                    
            self.tf[i]=conteo

        if len(res_dic)!=0:
            res_dic=list(set(res_dic))
            indx=[self.dicc.get(i,0) for i in res_dic]
            self.tf=self.tf[:,indx]
            self.tf_idf_keys={i:x for x,i in enumerate(res_dic)}

        self.get_idf()
        self.get_tf_idf()
            
    def get_vec(self, case, files= None):
        if files==None:
            fil=list(self.files.values())
        else:
            fil=[self.files.get(x,0) for x in files]
        match case:
            case 'tf':
                vector=np.mean(self.tf[fil,:],axis=0)
            case 'tf-idf':
                vector=np.mean(self.tf_idf[fil,:],axis=0)

        return vector
    
    def new_get_vec(self,file,mode='tf'):
        dicc=dict.fromkeys(self.tf_idf_keys.keys(),0)
        for word in file:
            if word in dicc.keys():
                dicc[word]=dicc.get(word,0)+1
        dicc=np.array(list(dicc.values()))
        match mode:
            case 'tf':
                return dicc
            case 'tf-idf':
                tf_idf=dicc*np.log2(self.n_doc/self.idf)
                return tf_idf
    
    def sim_cor(self,file,vec_files,mode='tf'):
        vect=self.get_vec(mode,vec_files)
        comp=self.new_get_vec(file, mode)
        distance = cosine(comp,vect)
        return distance 

    def show_topn(self,topn,files=None):
        if files==None:
            fil=list(self.files.values())
        else:
            fil=[self.files.get(x,0) for x in files]
        



## Cargo archivos, separo grupos

In [4]:
compendio={}
articulos=["articulos/"+ruta for ruta in os.listdir('articulos')]

for articulo in articulos:
    with open(articulo,"r",encoding="utf-8") as b:
        strp=b.read().lower()
    strp=prepro(strp)
    compendio[articulo.replace("articulos/",'')]=strp


In [5]:
test_lebron=compendio.pop('NBA.txt')
test_mgs=compendio.pop('Videojuego_de_sigilo.txt')


In [6]:
lebron=['Pie_(unidad).txt',
        'Ohio.txt',
        'Akron.txt',
        'Baloncesto.txt',
        'Cleveland_Cavaliers.txt',
        'Draft_de_la_NBA.txt',
        'Draft_de_la_NBA_de_2003.txt',
        'Estados_Unidos.txt',
        'Kilogramo.txt',
        'LeBron_James.txt']

mgs=['Entertainment_Software_Rating_Board.txt',
     'Hideo_Kojima.txt',
     'Konami.txt',
     'Metal_Gear_Solid.txt',
     'Metal_Gear_Solid_Digital_Graphic_Novel.txt',
     'Metal_Gear_Solid__The_Twin_Snakes.txt',
     'Microsoft.txt',
     'Pan_European_Game_Information.txt',
     'Videojuego_de_t%C3%A1ctica_en_tiempo_real.txt',
     'Videojuego_de_un_jugador.txt']

## Pruebas y vectores

#### TF, TF-IDF

In [7]:
vector = Vector(compendio)
vector.get_tf(mode='topn',topn=10) 

In [8]:
vector.tf_idf_keys

{'comic': 0,
 'incluye': 1,
 'twin': 2,
 'videojuegos': 3,
 'metal': 4,
 'aproximadamente': 5,
 'estadounidenses': 6,
 'cavaliers': 7,
 'medida': 8,
 'original': 9,
 'lebron': 10,
 'km²': 11,
 'paises': 12,
 'historia': 13,
 'mayor': 14,
 'association': 15,
 '2': 16,
 'estadounidense': 17,
 'software': 18,
 'dado': 19,
 'puntos': 20,
 'windows': 21,
 'mejores': 22,
 'batalla': 23,
 'modo': 24,
 'agregada': 25,
 'clasificacion': 26,
 '2007': 27,
 'goma': 28,
 'videojuego': 29,
 'eleccion': 30,
 'reglas': 31,
 'bosh': 32,
 'jugador': 33,
 'coherentes': 34,
 'nombre': 35,
 'conferencia': 36,
 'practicas': 37,
 'jugabilidad': 38,
 'draft': 39,
 'tambien': 40,
 'edades': 41,
 'gamecube': 42,
 'compañia': 43,
 '000': 44,
 'equipo': 45,
 'ciudad': 46,
 'personas': 47,
 'primera': 48,
 'mm': 49,
 'ganador': 50,
 'nintendo': 51,
 'esrb': 52,
 'parte': 53,
 'sistema': 54,
 'fabricacion': 55,
 'pegi': 56,
 'gear': 57,
 'version': 58,
 'unidos': 59,
 'snakes': 60,
 'si': 61,
 'tactica': 62,
 'cons

#### W2V

In [9]:
from gensim.models import Word2Vec as w2v
def article_vector(article):
    vectors = [modelo.wv[word] for word in article if word in modelo.wv]
    return np.mean(vectors, axis=0)

In [12]:
sentences=list(compendio.values())
modelo = w2v(sentences=sentences,vector_size=100,window=5,min_count=5,epochs=5,workers=6)

test_lebron_vector=article_vector(test_lebron)
test_mgs_vector=article_vector(test_mgs)

grupo_lebron={k:v for k,v in compendio.items() if k in lebron}
grupo_mgs={k:v for k,v in compendio.items() if k in mgs}

# Obtener representación vectorial de los artículos en grupo1 y grupo2
grupo_lebron_vectors = np.mean([article_vector(article) for article in grupo_lebron.values()],axis=0)
grupo_mgs_vectors = np.mean([article_vector(article) for article in grupo_mgs.values()],axis=0)

#### Testeo

<img src="https://i1.sndcdn.com/artworks-9vp6lBh9T65XVu1X-xMPTjg-t500x500.jpg" alt="drawing" width="200"/> <img src="https://w7.pngwing.com/pngs/867/35/png-transparent-arrow-right-face-funny-expressions-blue-pointing-pointers-directions-happy.png" alt="drawing" width="200"/> <img src="https://media.tenor.com/qBNk0a044ogAAAAe/metal-gear-solid-otacon.png" alt="drawing" width="300"/>


**Lebron** en **Lebron**

In [19]:
print(f"Distancia coseno (TF): {vector.sim_cor(test_lebron,vec_files=lebron,mode='tf')}")
print(f"Distancia coseno (TF-IDF): {vector.sim_cor(test_lebron,vec_files=lebron,mode='tf-idf')}")

similarity_grupo_lebron = cosine(test_lebron_vector, grupo_lebron_vectors) 
print("Distancia coseno (w2v):", similarity_grupo_lebron)

Distancia coseno (TF): 0.4407553428637969
Distancia coseno (TF-IDF): 0.5529962597297646
Distancia coseno (w2v): 3.74631375477108e-06


**Lebron** en **Metal Gear**

In [20]:
print(f"Distancia coseno (TF): {vector.sim_cor(test_lebron,vec_files=mgs,mode='tf')}")
print(f"Distancia coseno (TF-IDF): {vector.sim_cor(test_lebron,vec_files=mgs,mode='tf-idf')}")
similarity_grupo_lebron = cosine(test_lebron_vector, grupo_mgs_vectors) 
print("Distancia coseno (w2v)", similarity_grupo_lebron)

Distancia coseno (TF): 0.8657530251484995
Distancia coseno (TF-IDF): 0.9761068460012565
Distancia coseno (w2v) 2.2601483992357174e-05


**Metal Gear** en **Lebron**

In [21]:
print(f"Distancia coseno (TF): {vector.sim_cor(test_mgs,vec_files=lebron,mode='tf')}")
print(f"Distancia coseno (TF-IDF): {vector.sim_cor(test_mgs,vec_files=lebron,mode='tf-idf')}")

similarity_grupo_mgs = cosine(test_mgs_vector, grupo_lebron_vectors) 
print("Distancia coseno (w2v):", similarity_grupo_mgs)

Distancia coseno (TF): 0.7870840588061763
Distancia coseno (TF-IDF): 0.9421443221336212
Distancia coseno (w2v): 3.482735078941257e-05


**Metal Gear** en **Metal Gear**

In [22]:
print(f"Distancia coseno (TF): {vector.sim_cor(test_mgs,vec_files=mgs,mode='tf')}")
print(f"Distancia coseno (TF-IDF): {vector.sim_cor(test_mgs,vec_files=mgs,mode='tf-idf')}")

similarity_grupo_mgs = cosine(test_mgs_vector, grupo_mgs_vectors) 
print("Distancia coseno (w2v):", similarity_grupo_mgs)

Distancia coseno (TF): 0.36891613827363356
Distancia coseno (TF-IDF): 0.53555007254139
Distancia coseno (w2v): 8.984553926394412e-06


## Interpretacion

Intente colocar una al lado de otra, pero no pude (aun no se interpretar)

| TF                | Lebron   | MGS   |
| :---------------- | :------: | ----: |
| Lebron            |   0.44   | 0.86 |
| MGS               |   0.79   | 0.37 |



| TF-IDF            | Lebron   | MGS   |
| :---------------- | :------: | ----: |
| Lebron            |   0.55   | 0.98 |
| MGS               |   0.94   | 0.54 |





| W2V                | Lebron   | MGS   |
| :---------------- | :------: | ----: |
| Lebron            |   3.74e-06  | 2.26e-05 |
| MGS               |   3.48e-05   | 8.98e-06 |
